In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from google.colab import files

def normalize_ev_data():
    # Upload the CSV file
    print("Please upload your dataset:")
    # uploaded = files.upload()

    # Assume the uploaded file is the first in the dictionary
    csv_path = '/content/Electric_Vehicle_Population_Data (1).csv'

    # Read the CSV file
    df = pd.read_csv(csv_path)

    # Count missing values in the original dataset
    missing_values = df.isnull().sum()
    print("\nMissing values in original dataset:")
    print(missing_values)

    # Print column names to inspect
    print("\nAvailable columns in the dataset:")
    print(df.columns.tolist())

    # Clean up column names
    df.columns = df.columns.str.strip()

    # Map expected column names to actual column names
    column_mapping = {
        'Model Year': 'Model Year',
        'Electric Range': 'Electric Range',
        'Base MSRP': 'Base MSRP',
        'County': 'County',
        'City': 'City',
        'State': 'State',
        'Make': 'Make',
        'Model': 'Model',
        'Electric Vehicle Type': 'Electric Vehicle Type',
        'Clean Alternative Fuel Vehicle (CAFV) Eligibility': 'Clean Alternative Fuel Vehicle (CAFV) Eligibility',
        'Electric Utility': 'Electric Utility',
        'Legislative District': 'Legislative District',
        'Vehicle Location': 'Vehicle Location',
        'VIN (1-10)': 'VIN (1-10)',
        'Postal Code': 'Postal Code',
        '2020 Census Tract': '2020 Census Tract'
    }

    # Convert numeric columns
    numeric_columns = ['Model Year', 'Electric Range', 'Base MSRP']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Handle missing values
    df = df.fillna(df.median(numeric_only=True))  # Fill numeric missing values with median
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].fillna('Unknown')  # Fill categorical missing values with 'Unknown'

    # Count missing values in the dataset after initial handling
    missing_values = df.isnull().sum()
    print("\nMissing values after initial handling:")
    print(missing_values)

    # Handle categorical variables using Label Encoding
    categorical_columns = ['County', 'City', 'State', 'Make', 'Model',
                           'Electric Vehicle Type', 'Clean Alternative Fuel Vehicle (CAFV) Eligibility',
                           'Electric Utility', 'Legislative District']

    # Filter categorical columns that exist in the dataset
    existing_categorical_columns = [col for col in categorical_columns if col in df.columns]

    label_encoders = {}
    for col in existing_categorical_columns:
        label_encoders[col] = LabelEncoder()
        df[col] = label_encoders[col].fit_transform(df[col].astype(str))

    # Extract coordinates from Vehicle Location if it exists
    if 'Vehicle Location' in df.columns:
        df['Longitude'] = df['Vehicle Location'].str.extract(r'POINT \(([-\d.]+)')
        df['Latitude'] = df['Vehicle Location'].str.extract(r'([-\d.]+)\)')
        df['Longitude'] = pd.to_numeric(df['Longitude'])
        df['Latitude'] = pd.to_numeric(df['Latitude'])

        # Handle missing values in Longitude and Latitude
        longitude_median = df['Longitude'].median(skipna=True)
        latitude_median = df['Latitude'].median(skipna=True)

        df['Longitude'] = df['Longitude'].fillna(longitude_median)  # Replace missing with median
        df['Latitude'] = df['Latitude'].fillna(latitude_median)    # Replace missing with median

        # Drop the Vehicle Location column
        df = df.drop('Vehicle Location', axis=1)

    # Initialize MinMaxScaler
    scaler = MinMaxScaler()

    # Identify numeric columns for normalization
    numeric_columns_to_normalize = [col for col in ['Model Year', 'Electric Range',
                                                    'Longitude', 'Latitude'] if col in df.columns]
    columns_to_normalize = numeric_columns_to_normalize + existing_categorical_columns

    # Create a copy of the DataFrame with only the columns to normalize
    df_to_normalize = df[columns_to_normalize].copy()

    # Normalize the selected columns
    normalized_data = scaler.fit_transform(df_to_normalize)

    # Create new DataFrame with normalized data
    normalized_df = pd.DataFrame(normalized_data, columns=columns_to_normalize)

    # Save mapping dictionaries for future reference
    mapping_dict = {}
    for col in existing_categorical_columns:
        mapping_dict[col] = dict(zip(label_encoders[col].classes_,
                                     label_encoders[col].transform(label_encoders[col].classes_)))

    # Drop the unnecessary columns
    drop_columns = ['VIN (1-10)', 'Postal Code', '2020 Census Tract', 'Base MSRP']
    normalized_df = normalized_df.drop(columns=[col for col in drop_columns if col in df.columns], errors='ignore')

    return normalized_df, mapping_dict

def save_normalized_data(normalized_df, mapping_dict):
    # Save normalized data to a CSV file
    normalized_csv_path = 'ev_data_normalized.csv'
    normalized_df.to_csv(normalized_csv_path, index=False)
    print(f"Normalized data saved to {normalized_csv_path}")

    # Save mapping dictionary to a text file
    mapping_txt_path = 'ev_data_mappings.txt'
    with open(mapping_txt_path, 'w') as f:
        for col, mapping in mapping_dict.items():
            f.write(f"{col} mapping:\n")
            for key, value in mapping.items():
                f.write(f"  {key}: {value}\n")
            f.write("\n")
    print(f"Mapping dictionary saved to {mapping_txt_path}")

# Example usage
normalized_df, mapping_dict = normalize_ev_data()
save_normalized_data(normalized_df, mapping_dict)

# Download the normalized CSV file and mappings
files.download('ev_data_normalized.csv')
files.download('ev_data_mappings.txt')

# Print shape of normalized dataset
print("\nShape of normalized dataset:", normalized_df.shape)
print("\nColumns in normalized dataset:", normalized_df.columns.tolist())

# Display first few rows of normalized data
print("\nFirst few rows of normalized data:")
print(normalized_df.head())


Please upload your dataset:

Missing values in original dataset:
VIN (1-10)                                             0
County                                                 3
City                                                   3
State                                                  0
Postal Code                                            3
Model Year                                             0
Make                                                   0
Model                                                  0
Electric Vehicle Type                                  0
Clean Alternative Fuel Vehicle (CAFV) Eligibility      0
Electric Range                                         0
Base MSRP                                              0
Legislative District                                 463
DOL Vehicle ID                                         0
Vehicle Location                                       9
Electric Utility                                       3
2020 Census Tract      

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Shape of normalized dataset: (220225, 13)

Columns in normalized dataset: ['Model Year', 'Electric Range', 'Longitude', 'Latitude', 'County', 'City', 'State', 'Make', 'Model', 'Electric Vehicle Type', 'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Utility', 'Legislative District']

First few rows of normalized data:
   Model Year  Electric Range  Longitude  Latitude    County      City  \
0    0.807692        0.789318   0.410810  0.664451  0.427885  0.796935   
1    0.961538        0.115727   0.411335  0.664644  0.427885  0.796935   
2    0.961538        0.115727   0.411096  0.662369  0.427885  0.796935   
3    0.730769        0.637982   0.412144  0.664532  0.427885  0.429119   
4    0.500000        0.103858   0.404858  0.647698  0.903846  0.628352   

      State      Make     Model  Electric Vehicle Type  \
0  0.957447  0.866667  0.569620                    0.0   
1  0.957447  0.111111  0.962025                    1.0   
2  0.957447  0.111111  0.962025               